In [28]:
import subprocess
import numpy as np
import pandas as pd
from multiprocess import Pool
from itertools import product
from math import prod as mul
from iteration_utilities import random_combination
from tqdm.notebook import tqdm
from random import seed, randrange



In [29]:
s_block = [8, 0, 12, 4, 9, 6, 7, 11, 2, 3, 1, 15, 5, 14, 10, 13]

s_block_rev = [1, 10, 8, 9, 3, 12, 5, 6, 0, 4, 14, 7, 2, 13, 15, 11]

list_size = 1 << 16


In [30]:
class heys_cipher:
	def substitution(self, number, need_cipher = True):
		new_number = 0

		operation = s_block if need_cipher == True else s_block_rev

		for i in range(4):
			new_number |= operation[(number >> i * 4) & 15] << i * 4

		return new_number
	
	def mix_words(self, number):
		new_number = 0

		for j in range(4):
			for i in range(4):
				new_number |= (number >> (4 * j + i) & 1) << (4 * i + j)

		return new_number

	def encrypt(self, number, key = 0):
		new_number = number ^ key
		new_number = self.substitution(new_number)
		new_number = self.mix_words(new_number)

		return new_number

	def decrypt(self, number, key = 0):
		new_number = self.mix_words(new_number)
		new_number = self.substitution(new_number, False)
		new_number = number ^ key

		return new_number


In [31]:
heys_instance = heys_cipher()

def encrypt(number, key):
	round_keys = []

	for i in range(7):
		round_keys.append((key >> 16 * i) & 65535)

	encrypted_number = number

	for i in range(6):
		encrypted_number = heys_instance.encrypt(encrypted_number, round_keys[i])

	encrypted_number ^= round_keys[6]

	return encrypted_number


In [32]:
def scalar_multiplication(a, b):
	return (a & b).bit_count() & 1


In [33]:
lin_s_block_table = np.zeros((16, 16))

for alpha in range(16):
	for beta in range(16):
		lin_s_block_table[alpha][beta] = (sum([
			(-1) ** (
				scalar_multiplication(alpha, x) ^ scalar_multiplication(beta, s_block[x])
			) for x in range(16)
		]) / 16) ** 2

pd.DataFrame(lin_s_block_table)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,0.0,0.0000,0.0625,0.0625,0.0625,0.0625,0.0000,0.2500,0.0000,0.2500,0.0625,0.0625,0.0625,0.0625,0.0000,0.0000
2,0.0,0.0625,0.0000,0.0625,0.0625,0.0000,0.0625,0.2500,0.0625,0.0000,0.0625,0.0000,0.0000,0.0625,0.2500,0.0625
3,0.0,0.0625,0.0625,0.0000,0.0000,0.0625,0.0625,0.0000,0.0625,0.2500,0.0000,0.0625,0.0625,0.0000,0.2500,0.0625
4,0.0,0.0625,0.0625,0.2500,0.0625,0.0000,0.0000,0.0625,0.0625,0.0000,0.0000,0.0625,0.2500,0.0625,0.0625,0.0000
5,0.0,0.0625,0.0000,0.0625,0.0000,0.0625,0.0000,0.0625,0.0625,0.0000,0.0625,0.2500,0.0625,0.0000,0.0625,0.2500
6,0.0,0.0000,0.0625,0.0625,0.2500,0.2500,0.0625,0.0625,0.0000,0.0000,0.0625,0.0625,0.0000,0.0000,0.0625,0.0625
7,0.0,0.2500,0.2500,0.0000,0.0625,0.0625,0.0625,0.0625,0.0000,0.0000,0.0000,0.0000,0.0625,0.0625,0.0625,0.0625
8,0.0,0.0625,0.0625,0.2500,0.0000,0.0625,0.0625,0.0000,0.0000,0.0625,0.0625,0.0000,0.2500,0.0625,0.0625,0.0000
9,0.0,0.0625,0.0000,0.0625,0.0625,0.0000,0.0625,0.0000,0.2500,0.0625,0.2500,0.0625,0.0625,0.0000,0.0625,0.0000


In [34]:
def compute_temporary_list(alpha):
	alpha_array = [(alpha >> 4 * i) & 15 for i in range(4)]

	not_null_betas = [
		[
			beta for beta in range(16) if lin_s_block_table[alpha_array[i]][beta] != 0
		] for i in range(4)
	]

	lin_list = {}

	for (i, beta_array) in enumerate(product(*not_null_betas)):
		beta = sum([beta_array[i] << i * 4 for i in range(4)])

		lin_list[beta] = mul([lin_s_block_table[alpha_array[i]][beta_array[i]] for i in range(4)])

	return lin_list


In [35]:
p = 0.00015

def linear_search(alpha):
	lin_table = {}

	list_prev = {alpha: 1}

	for i in range (1, 6):
		list_curr = {}

		for beta in list_prev:
			p_i = list_prev[beta]

			if beta not in lin_table:
				lin_table[beta] = compute_temporary_list(beta)

			for gamma in lin_table[beta]:
				additional_probability = p_i * lin_table[beta][gamma]	

				list_curr[gamma] = additional_probability if gamma not in list_curr else list_curr[gamma] + additional_probability

		list_new_curr = {}

		for gamma in list_curr.keys():
			if list_curr[gamma] > p:
				list_new_curr[heys_instance.mix_words(gamma)] = list_curr[gamma]

		list_prev = list_new_curr

	return (alpha, list_prev)


In [36]:
with Pool() as pool:
	candidates = np.array(
		pool.map(
			linear_search,
			[(j << 4 * i) for i in range(4) for j in range(1, 16)]
		)
	)

len(candidates)


60

In [37]:
candidates

array([[1, {}],
       [2, {}],
       [3, {136: 0.0001576840877532959}],
       [4,
        {273: 0.0001506507396697998, 2184: 0.000246446201344952, 34944: 0.0001682024449110031, 32776: 0.00016705095185898244}],
       [5, {}],
       [6,
        {128: 0.0001500739308539778, 136: 0.00017753243446350098, 2184: 0.00016156240599229932, 32776: 0.00018283905228599906, 34816: 0.00018365998403169215, 34944: 0.0001691259676590562}],
       [7,
        {17: 0.00017979508265852928, 68: 0.00015866011381149292, 136: 0.0002516050008125603, 2184: 0.00019286572933197021, 4352: 0.00016719987615942955, 34816: 0.0002316225436516106}],
       [8, {2184: 0.0001900196075439453}],
       [9,
        {8: 0.00015442210133187473, 2184: 0.00020836250041611493, 34944: 0.00020475441124290228, 128: 0.000150190171552822, 32776: 0.00019806335330940783}],
       [10, {34816: 0.00015764092677272856}],
       [11,
        {4097: 0.0001527991844341159, 128: 0.00017107173334807158, 136: 0.00016638636589050293, 2184: 0.0

In [38]:
approximations = []

for alpha, betas in list(candidates):
	for beta in betas.keys():
		approximations.append(((alpha, beta), betas[beta]))

approximations = sorted(approximations, key = lambda x: x[1], reverse = True)

print(len(approximations))

approximations[:25]


360


[((1024, 546), 0.00037679132947232574),
 ((1024, 273), 0.00037679132947232574),
 ((3072, 8704), 0.00037357212659117067),
 ((3072, 4352), 0.00037357212659117067),
 ((3072, 34), 0.0003700285597005859),
 ((3072, 17), 0.0003700285597005859),
 ((1792, 34), 0.00034421173040755093),
 ((1792, 17), 0.00034421173040755093),
 ((1792, 8704), 0.00033912872731889365),
 ((1792, 4352), 0.00033912872731889365),
 ((2304, 8736), 0.0003341819974593818),
 ((2304, 4368), 0.0003341819974593818),
 ((2816, 8194), 0.0003175786987412721),
 ((2816, 4097), 0.0003175786987412721),
 ((3840, 8194), 0.0003158958425046876),
 ((3840, 4097), 0.0003158958425046876),
 ((3840, 546), 0.0003138675383524969),
 ((3840, 273), 0.0003138675383524969),
 ((2304, 546), 0.000313625336275436),
 ((2304, 273), 0.000313625336275436),
 ((2304, 8194), 0.0003072636463912204),
 ((2304, 4097), 0.0003072636463912204),
 ((3840, 8736), 0.000304335611872375),
 ((3840, 4368), 0.000304335611872375),
 ((2048, 546), 0.000298425555229187)]

In [39]:
unique_alpha = set([approximation[0] for approximation, potential in approximations])
unique_beta = set([approximation[1] for approximation, potential in approximations])

scalar_multiplication_array = dict([(i, np.array([scalar_multiplication(i, j) for j in range(list_size)], dtype = bool)) for i in unique_alpha.union(unique_beta)])


In [40]:
def write_texts(file, texts):
	with open(file, 'wb') as f:
		for text in texts:
			f.write(text.to_bytes(2, 'little'))


In [41]:
def read_texts(file):

	with open(file, 'rb') as f:
		line = f.read()
		concatenated_text = int.from_bytes(line, byteorder='little')

		text_count = len(line) // 2
		texts = np.zeros(text_count, dtype = np.uint16)

		for i in range(text_count):
			texts[i] = ((concatenated_text >> (i * 16)) & (list_size - 1))

	return texts


In [42]:
input = './input.bin'
output = './output.bin'

def create_statistical_materials(count):
	random.seed(5)

	plain_texts = random_combination(range(list_size), r = count)
	write_texts(input, plain_texts)

	subprocess.run(f'./heys.bin e 02 {input} {output}', shell = True)
	cipher_texts = read_texts(output)

	with Pool() as pool:
		texts_after_first_round = np.array(
			pool.map(
				lambda key: np.array(
					[heys_instance.mix_words(heys_instance.substitution(x ^ key)) for x in plain_texts],
					dtype = np.uint16
				), 
	    		range(list_size)
			)
		)

	return texts_after_first_round, cipher_texts


In [47]:
texts_count = 6000

In [48]:
%%time

texts_after_first_round, cipher_texts = create_statistical_materials(texts_count)


* Heys cipher version 0.1 *
Encrypt file "./input.bin" to file "./output.bin"...
done.

Press <Enter>...
CPU times: user 1.84 s, sys: 744 ms, total: 2.58 s
Wall time: 2min 42s


In [49]:
def check_key(texts_after_first_round, cipher_texts, alpha, beta):
	unos_count = 0

	for x, y in zip(texts_after_first_round, cipher_texts):
		unos_count += scalar_multiplication_array[alpha][x] ^ scalar_multiplication_array[beta][y]

	return abs(texts_count - 2 * unos_count)


In [50]:
%%time

seed(randrange(list_size))
keys_candidates = {}

for index, (approximation, probability) in tqdm(enumerate(approximations), total = len(approximations), desc = "Processing"):
	alpha, beta = approximation

	with Pool() as pool:
		temp = np.array(
			pool.map(
				lambda key: check_key(texts_after_first_round[key], cipher_texts, alpha, beta),
				range(list_size)
			),
			dtype = np.uint16
		)

		count = 100
		keys_candidates_with_max_u = []
		
		for max_u in range(max(temp), 0, -1):
			keys_with_max_u = [key for key, u in enumerate(temp) if u == max_u]

			length = len(keys_with_max_u)

			if length <= count:
				keys_candidates_with_max_u += keys_with_max_u
				count -= length
			else:
				keys_candidates_with_max_u += random_combination(keys_with_max_u, r = count)
				break

		keys_candidates[approximation] = np.array(keys_candidates_with_max_u, dtype = np.uint16)


Processing:   0%|          | 0/360 [00:00<?, ?it/s]

CPU times: user 10min 51s, sys: 53.6 s, total: 11min 45s
Wall time: 1h 47min 3s


In [51]:
sorted_key_candidates = {}

for approximation, keys in keys_candidates.items():
    for key in keys:
        if key in sorted_key_candidates:
            sorted_key_candidates[key] += 1
        else:
            sorted_key_candidates[key] = 1
            
sorted_key_candidates = sorted(sorted_key_candidates.items(), key = lambda x: x[1], reverse = True)

In [52]:
[(hex(key), count) for key, count in sorted_key_candidates[:10]]

[('0x4be8', 7),
 ('0x4b58', 6),
 ('0x4202', 6),
 ('0x4c0c', 5),
 ('0x776', 5),
 ('0x876', 5),
 ('0x57aa', 5),
 ('0x5caa', 5),
 ('0xfc7b', 5),
 ('0x49e8', 5)]